In [1]:
import pandas as pd
import requests
import boto3
import ast
import datetime
import time

In [2]:
from cobinhood_api import Cobinhood
cob = Cobinhood()

print(cob.system.get_time())

2018-09-29 12:30:25 chriss-mbp.lan cobinhood-api[4256] DEBUG cobinhood_api.http.system fetch "get_time"


{'success': True, 'result': {'time': 1538238625875}}


In [3]:
cob.market.get_trading_pairs()

2018-07-27 07:16:56 Chriss-MacBook-Pro.local cobinhood-api[4018] DEBUG cobinhood_api.http.market fetch "get_trading_pairs"


{'success': True,
 'result': {'trading_pairs': [{'id': 'ABT-BTC',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'BTC',
    'base_max_size': '2516723.629',
    'base_min_size': '50.335',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'ABT-ETH',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'ETH',
    'base_max_size': '2516723.629',
    'base_min_size': '50.335',
    'quote_increment': '0.0000001',
    'margin_enabled': False},
   {'id': 'ACT-BTC',
    'base_currency_id': 'ACT',
    'quote_currency_id': 'BTC',
    'base_max_size': '11934470.211',
    'base_min_size': '238.69',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-BTC',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'BTC',
    'base_max_size': '2991280.418',
    'base_min_size': '59.826',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-ETH',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'ETH',

In [3]:
# Instantiate resources
try:
    boto_session = boto3.Session(profile_name='loidsig')
except:
    boto_session = boto3.Session()
s3_resource = boto_session.resource('s3')
s3_bucket = 'loidsig-crypto'

sm_client = boto_session.client(
    service_name='secretsmanager',
    region_name='us-east-1',
    endpoint_url='https://secretsmanager.us-east-1.amazonaws.com'
)
get_secret_value_response = sm_client.get_secret_value(SecretId='Loidsig_CPM_Cobinhood')
key, value = ast.literal_eval(get_secret_value_response['SecretString']).popitem()
cob = Cobinhood(API_TOKEN=value)

In [5]:
print(cob.wallet.get_balances())

2018-08-14 06:12:23 Chriss-MacBook-Pro.local cobinhood-api[1032] DEBUG cobinhood_api.http.wallet fetch "get_balances"


{'success': True, 'result': {'balances': [{'currency': 'PLY', 'type': 'exchange', 'total': '2', 'on_order': '0', 'locked': False, 'usd_value': '0.231544796', 'btc_value': '0.00003836'}, {'currency': 'SHPING', 'type': 'exchange', 'total': '1', 'on_order': '0', 'locked': False, 'usd_value': '0.000482888', 'btc_value': '0.00000008'}, {'currency': 'HQX', 'type': 'exchange', 'total': '1', 'on_order': '0', 'locked': False, 'usd_value': '0.008035278', 'btc_value': '0.00000133'}, {'currency': 'FXT', 'type': 'exchange', 'total': '1', 'on_order': '0', 'locked': False, 'usd_value': '0.03199133', 'btc_value': '0.0000053'}, {'currency': 'USDT', 'type': 'exchange', 'total': '2.4501111324', 'on_order': '0', 'locked': False, 'usd_value': '2.46005160715110560424', 'btc_value': '0.000406448935753836'}, {'currency': 'ETH', 'type': 'exchange', 'total': '1.48', 'on_order': '0', 'locked': False, 'usd_value': '395.04756842856', 'btc_value': '0.0652715224'}]}}


### Get usd wallet balance

In [11]:
wallet_dict = cob.wallet.get_balances()['result']['balances']
wallet_df = pd.DataFrame(wallet_dict)
wallet_df.rename(columns={'total':'quantity', 'currency':'coin'}, inplace=True)
wallet_df['access_datetime'] = datetime.datetime.utcfromtimestamp(time.time()).strftime('%Y-%m-%d %I:%M:%S')
wallet_df

2018-08-14 06:21:10 Chriss-MacBook-Pro.local cobinhood-api[1032] DEBUG cobinhood_api.http.wallet fetch "get_balances"


,btc_value,coin,locked,on_order,quantity,type,usd_value,access_datetime
0,0.00003836,PLY,False,0,2,exchange,0.23257668,2018-08-14 10:21:10
1,0.00000008,SHPING,False,0,1,exchange,0.00048504,2018-08-14 10:21:10
2,0.00000133,HQX,False,0,1,exchange,0.0080973225,2018-08-14 10:21:10
3,0.0000053,FXT,False,0,1,exchange,0.0321339,2018-08-14 10:21:10
4,0.000404292837957324,USDT,False,0,2.4501111324,exchange,2.45326382490516024876,2018-08-14 10:21:10
5,0.0654283728,ETH,False,0,1.48,exchange,397.02444265548,2018-08-14 10:21:10


In [16]:
from sqlalchemy import create_engine
def logic_db_engine():
    """Fetches Logic DB postgres connection object

    Returns:
        A database connection object for Postgres
    """
    try:
        boto_session = boto3.Session(profile_name='loidsig')
    except:
        boto_session = boto3.Session()
    sm_client = boto_session.client(
        service_name='secretsmanager',
        region_name='us-east-1',
        endpoint_url='https://secretsmanager.us-east-1.amazonaws.com'
    )
    get_secret_value_response = sm_client.get_secret_value(SecretId='Loidsig_DB')
    cred_dict = ast.literal_eval(get_secret_value_response['SecretString'])
    db_user, db_pass = cred_dict['username'], cred_dict['password']
    db_host, db_port, db_name = cred_dict['host'], cred_dict['port'], cred_dict['dbname']
    print(db_user)

    try:
        postgres_engine = create_engine(f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')
    except Exception as e:
        print("Unable to connect to postgres! Error: {}".format(e))
        raise
    return postgres_engine

In [18]:
wallet_df.to_sql('cobinhood', logic_db_engine(), schema='wallet')

the_logic


In [4]:
cob.market.get_stats()

2018-09-08 14:25:29 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_stats"


{'success': True,
 'result': {'ABT-BTC': {'id': 'ABT-BTC',
   'last_price': '0',
   'lowest_ask': '0.000043',
   'highest_bid': '0.00003112',
   'base_volume': '0',
   'quote_volume': '0',
   'is_frozen': False,
   'high_24hr': '0',
   'low_24hr': '0',
   'percent_changed_24hr': '0'},
  'ABT-ETH': {'id': 'ABT-ETH',
   'last_price': '0.0016',
   'lowest_ask': '0.0016',
   'highest_bid': '0.0011807',
   'base_volume': '3411.02426764',
   'quote_volume': '5.3297612377454',
   'is_frozen': False,
   'high_24hr': '0.0016',
   'low_24hr': '0.00146',
   'percent_changed_24hr': '0.0958904109589041'},
  'ACT-BTC': {'id': 'ACT-BTC',
   'last_price': '0',
   'lowest_ask': '0.00000724',
   'highest_bid': '0.00000532',
   'base_volume': '0',
   'quote_volume': '0',
   'is_frozen': False,
   'high_24hr': '0',
   'low_24hr': '0',
   'percent_changed_24hr': '0'},
  'BAT-BTC': {'id': 'BAT-BTC',
   'last_price': '0.000029',
   'lowest_ask': '0.000029',
   'highest_bid': '0.00001985',
   'base_volume': '

In [8]:
cob.market.get_tickers('BTC-USDT')['result']['ticker']['last_trade_price']

2018-09-08 14:40:12 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_tickers"


'6156.6'

In [ ]:
cob.market.get_tickers(self.formatted_target_coin)['result']['ticker']['last_trade_price']

In [ ]:
float([data['last_price'] for coin_pair, data in self.cob_client.market.get_tickers()['result'].items() if data['id'] == self.formatted_target_coin][0])

### Candlestick data

In [20]:
min_ago = 4
prefix = 'btcusdt'
starting_unix_time = int((time.time() - (min_ago * 60)) * 1000)
ending_unix_time = int(time.time() * 1000)

cob_candles_list = cob.chart.get_candles(trading_pair_id='BTC-USDT', start_time=starting_unix_time, end_time=ending_unix_time, timeframe='1m')['result']['candles']
df = pd.DataFrame(cob_candles_list)
df.columns=[f'{prefix}_close',f'{prefix}_high',f'{prefix}_low',f'{prefix}_open','timeframe','close_timestamp','coin',f'{prefix}_volume']
df['coin'] = df['coin'].replace(regex=True, to_replace=r'-', value=r'')
df['close_timestamp_trim'] = df['close_timestamp']/1000
df['close_datetime'] = pd.to_datetime(df['close_timestamp_trim'], unit='s')
#df['minute'] = pd.to_numeric(df['close_timestamp']/1000/60).astype(int)
df

2018-08-28 13:09:30 Chriss-MacBook-Pro.local cobinhood-api[7536] DEBUG cobinhood_api.http.chart fetch "get_candles"


,btcusdt_close,btcusdt_high,btcusdt_low,btcusdt_open,timeframe,close_timestamp,coin,btcusdt_volume,close_timestamp_trim,close_datetime
0,NaN,NaN,NaN,NaN,1m,NaN,NaN,NaN,NaN,NaT
1,NaN,NaN,NaN,NaN,1m,NaN,NaN,NaN,NaN,NaT
2,NaN,NaN,NaN,NaN,1m,NaN,NaN,NaN,NaN,NaT
3,NaN,NaN,NaN,NaN,1m,NaN,NaN,NaN,NaN,NaT
4,NaN,NaN,NaN,NaN,1m,NaN,NaN,NaN,NaN,NaT


In [30]:
df = pd.DataFrame(cob_candles_list)
df

,close,high,low,open,timeframe,timestamp,trading_pair_id,volume
0,7016.3,7016.3,7016.3,7016.3,1m,1535475900000,BTC-USDT,0.22310439
1,7016.7,7016.7,7016.7,7016.7,1m,1535475960000,BTC-USDT,0.23982296
2,7014.4,7014.4,7014.4,7014.4,1m,1535476020000,BTC-USDT,0.31931768
3,7021.2,7021.2,7018.2,7018.2,1m,1535476080000,BTC-USDT,0.12113718
4,7021.2,7021.2,7021.2,7021.2,1m,1535476140000,BTC-USDT,0


### Placing an order

In [20]:
data = {
    "trading_pair_id": "ETH-USDT",
    "side": "bid",
    "type": "market",
    "size": "1.000"
}
result = cob.trading.post_orders(data)

2018-08-06 08:02:53 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.trading fetch "post_orders"


In [21]:
trade_id = result['result']['order']['id']
trade_id

'7b03bbf4-e67c-44ea-878c-11fe822462f0'

In [22]:
cob.trading.get_orders(trade_id)

2018-08-06 08:02:59 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.trading fetch "get_orders"


{'success': True,
 'result': {'order': {'id': '7b03bbf4-e67c-44ea-878c-11fe822462f0',
   'trading_pair_id': 'ETH-USDT',
   'side': 'bid',
   'type': 'market',
   'price': '0',
   'size': '1',
   'filled': '1',
   'state': 'filled',
   'timestamp': 1533556974069,
   'eq_price': '408.0537288676',
   'completed_at': '2018-08-06T12:02:54.095305855Z',
   'source': 'exchange'}}}

In [41]:
print(cob.wallet.get_balances())

2018-09-05 07:56:04 Chriss-MacBook-Pro.local cobinhood-api[7536] DEBUG cobinhood_api.http.wallet fetch "get_balances"


{'success': True, 'result': {'balances': [{'currency': 'PLY', 'type': 'exchange', 'total': '2', 'on_order': '0', 'locked': False, 'usd_value': '0.327684', 'btc_value': '0.000047'}, {'currency': 'SHPING', 'type': 'exchange', 'total': '1', 'on_order': '0', 'locked': False, 'usd_value': '0.00048804', 'btc_value': '0.00000007'}, {'currency': 'HQX', 'type': 'exchange', 'total': '1', 'on_order': '0', 'locked': False, 'usd_value': '0.0097241332', 'btc_value': '0.00000139'}, {'currency': 'FXT', 'type': 'exchange', 'total': '2', 'on_order': '0', 'locked': False, 'usd_value': '0.04713072', 'btc_value': '0.00000676'}, {'currency': 'USDT', 'type': 'exchange', 'total': '441.7613979324', 'on_order': '0', 'locked': False, 'usd_value': '444.56646079477263107112', 'btc_value': '0.063489948110844528'}, {'currency': 'ETH', 'type': 'exchange', 'total': '0', 'on_order': '0', 'locked': False, 'usd_value': '0', 'btc_value': '0'}, {'currency': 'COB', 'type': 'exchange', 'total': '2000', 'on_order': '0', 'lock

In [39]:
bid_trade_data = {
            "trading_pair_id": "BTC-USDT",
            "side": "bid",
            "type": "market",
            "size": ".05"
        }
buy_order_id = cob.trading.post_orders(bid_trade_data)
buy_order_id

2018-09-05 07:55:53 Chriss-MacBook-Pro.local cobinhood-api[7536] DEBUG cobinhood_api.http.trading fetch "post_orders"


{'success': True,
 'result': {'order': {'id': '65211e70-9ef6-489c-a931-c4d373c30b11',
   'trading_pair_id': 'BTC-USDT',
   'side': 'bid',
   'type': 'market',
   'price': '0',
   'size': '0.05',
   'filled': '0',
   'state': 'queued',
   'timestamp': 1536148553862,
   'eq_price': '0',
   'completed_at': None,
   'source': 'exchange'}}}

In [40]:
buy_order = cob.trading.get_orders(buy_order_id['result']['order']['id'])
buy_order

2018-09-05 07:55:59 Chriss-MacBook-Pro.local cobinhood-api[7536] DEBUG cobinhood_api.http.trading fetch "get_orders"


{'success': True,
 'result': {'order': {'id': '65211e70-9ef6-489c-a931-c4d373c30b11',
   'trading_pair_id': 'BTC-USDT',
   'side': 'bid',
   'type': 'market',
   'price': '0',
   'size': '0.05',
   'filled': '0.05',
   'state': 'filled',
   'timestamp': 1536148553862,
   'eq_price': '6988.5',
   'completed_at': '2018-09-05T11:55:53.882548811Z',
   'source': 'exchange'}}}

In [ ]:
buy_order_state = buy_order['result']['order']['state']

In [42]:
cob.trading.get_orders('c38f756d-74a3-4e44-8f53-4e4705d65bc9')

2018-09-07 07:12:10 Chriss-MacBook-Pro.local cobinhood-api[7536] DEBUG cobinhood_api.http.trading fetch "get_orders"


{'success': True,
 'result': {'order': {'id': 'c38f756d-74a3-4e44-8f53-4e4705d65bc9',
   'trading_pair_id': 'BTC-USDT',
   'side': 'bid',
   'type': 'market',
   'price': '0',
   'size': '0.01067',
   'filled': '0.01067',
   'state': 'filled',
   'timestamp': 1536317357275,
   'eq_price': '6377.7',
   'completed_at': '2018-09-07T10:49:17.29896Z',
   'source': 'exchange'}}}

In [43]:
cob.trading.get_orders('42838a69-2960-4766-8acb-911a9bdcd965')

2018-09-07 07:12:33 Chriss-MacBook-Pro.local cobinhood-api[7536] DEBUG cobinhood_api.http.trading fetch "get_orders"


{'success': True,
 'result': {'order': {'id': '42838a69-2960-4766-8acb-911a9bdcd965',
   'trading_pair_id': 'BTC-USDT',
   'side': 'ask',
   'type': 'market',
   'price': '0',
   'size': '0.01067',
   'filled': '0.01067',
   'state': 'filled',
   'timestamp': 1536317521129,
   'eq_price': '6365.4',
   'completed_at': '2018-09-07T10:52:01.153585Z',
   'source': 'exchange'}}}

### Other Data

In [27]:
cob.market.get_currencies()

2018-08-10 07:13:37 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_currencies"


{'success': True,
 'result': {'currencies': [{'currency': 'COB',
    'name': 'COBINHOOD Token',
    'type': 'erc20',
    'min_unit': '0.00000001',
    'deposit_fee': '0',
    'withdrawal_fee': '86.15',
    'min_withdrawal': '287.15392',
    'funding_min_size': '861.462',
    'interest_increment': '0.0001',
    'is_visible': True,
    'funding_enabled': True,
    'quotation_enabled': False,
    'required_kyc_level': 1,
    'deposit_frozen': False,
    'withdrawal_frozen': False,
    'cob_withdrawal_fee': '0'},
   {'currency': 'COFI',
    'name': 'CoinFi',
    'type': 'erc20',
    'min_unit': '0.00000001',
    'deposit_fee': '0',
    'withdrawal_fee': '144.15',
    'min_withdrawal': '480.49023',
    'funding_min_size': '1441.471',
    'interest_increment': '0.0001',
    'is_visible': True,
    'funding_enabled': False,
    'quotation_enabled': False,
    'required_kyc_level': 1,
    'deposit_frozen': False,
    'withdrawal_frozen': False,
    'cob_withdrawal_fee': '41.32972028'},
   {'cu

In [30]:
pairs = cob.market.get_trading_pairs()
pairs

2018-08-10 07:15:21 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_trading_pairs"


{'success': True,
 'result': {'trading_pairs': [{'id': 'ABT-BTC',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'BTC',
    'base_max_size': '4233908.184',
    'base_min_size': '84.679',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'ABT-ETH',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'ETH',
    'base_max_size': '4233908.184',
    'base_min_size': '84.679',
    'quote_increment': '0.0000001',
    'margin_enabled': False},
   {'id': 'ACT-BTC',
    'base_currency_id': 'ACT',
    'quote_currency_id': 'BTC',
    'base_max_size': '20444594.49',
    'base_min_size': '408.892',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-BTC',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'BTC',
    'base_max_size': '4063529.991',
    'base_min_size': '81.271',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-ETH',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'ETH',

In [14]:
pairs_payload = cob.market.get_trading_pairs()
pairs_list = [[pair['id'].lower().replace('-', ''), pair['id']] for pair in pairs_payload['result']['trading_pairs']]
pairs_dict = dict(pairs_list)
pairs_dict

2018-08-27 08:08:40 Chriss-MacBook-Pro.local cobinhood-api[7536] DEBUG cobinhood_api.http.market fetch "get_trading_pairs"


{'abtbtc': 'ABT-BTC',
 'abteth': 'ABT-ETH',
 'actbtc': 'ACT-BTC',
 'batbtc': 'BAT-BTC',
 'bateth': 'BAT-ETH',
 'bchbtc': 'BCH-BTC',
 'bcheth': 'BCH-ETH',
 'bchusdt': 'BCH-USDT',
 'bdgbtc': 'BDG-BTC',
 'bdgeth': 'BDG-ETH',
 'bdgusdt': 'BDG-USDT',
 'beebtc': 'BEE-BTC',
 'beeeth': 'BEE-ETH',
 'bmbtc': 'BM-BTC',
 'bmeth': 'BM-ETH',
 'boebtc': 'BOE-BTC',
 'boeeth': 'BOE-ETH',
 'botbtc': 'BOT-BTC',
 'boteth': 'BOT-ETH',
 'brdbtc': 'BRD-BTC',
 'brdeth': 'BRD-ETH',
 'brdusdt': 'BRD-USDT',
 'btcusdt': 'BTC-USDT',
 'cgcbtc': 'CGC-BTC',
 'cgceth': 'CGC-ETH',
 'cgcusdt': 'CGC-USDT',
 'cmtbtc': 'CMT-BTC',
 'cmtcob': 'CMT-COB',
 'cmteth': 'CMT-ETH',
 'cmtusdt': 'CMT-USDT',
 'cobbtc': 'COB-BTC',
 'cobeth': 'COB-ETH',
 'cobusdt': 'COB-USDT',
 'cofibtc': 'COFI-BTC',
 'cofieth': 'COFI-ETH',
 'dadibtc': 'DADI-BTC',
 'dadieth': 'DADI-ETH',
 'dashbtc': 'DASH-BTC',
 'dasheth': 'DASH-ETH',
 'dashusdt': 'DASH-USDT',
 'dentbtc': 'DENT-BTC',
 'denteth': 'DENT-ETH',
 'dgdbtc': 'DGD-BTC',
 'dgdeth': 'DGD-ETH',
 '

#### Get assset balance

In [44]:
float([balance for balance in cob.wallet.get_balances()['result']['balances'] if balance['currency'] == 'USDT'][0]['total'])

2018-08-10 07:32:53 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.wallet fetch "get_balances"


2.4501111324

In [43]:
[balance for balance in cob.wallet.get_balances()['result']['balances'] if balance['currency'] == 'USDT']

2018-08-10 07:30:33 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.wallet fetch "get_balances"


[{'currency': 'USDT',
  'type': 'exchange',
  'total': '2.4501111324',
  'on_order': '0',
  'locked': False,
  'usd_value': '2.45615722851946754976',
  'btc_value': '0.000383736405556488'}]

In [50]:
cob.market.get_stats()['result']

2018-08-10 07:37:55 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_stats"


{'ABT-BTC': {'id': 'ABT-BTC',
  'last_price': '0.00003',
  'lowest_ask': '0.000046',
  'highest_bid': '0.00002664',
  'base_volume': '223.29',
  'quote_volume': '0.0079407135611545',
  'is_frozen': False,
  'high_24hr': '0.00003566',
  'low_24hr': '0.00003',
  'percent_changed_24hr': '-0.1587212563095906'},
 'ABT-ETH': {'id': 'ABT-ETH',
  'last_price': '0',
  'lowest_ask': '0.000749',
  'highest_bid': '0.0005905',
  'base_volume': '0',
  'quote_volume': '0',
  'is_frozen': False,
  'high_24hr': '0',
  'low_24hr': '0',
  'percent_changed_24hr': '0'},
 'ACT-BTC': {'id': 'ACT-BTC',
  'last_price': '0.00000777',
  'lowest_ask': '0',
  'highest_bid': '0',
  'base_volume': '549',
  'quote_volume': '0.00426573',
  'is_frozen': False,
  'high_24hr': '0.00000777',
  'low_24hr': '0.00000777',
  'percent_changed_24hr': '0'},
 'BAT-BTC': {'id': 'BAT-BTC',
  'last_price': '0',
  'lowest_ask': '0.00004361',
  'highest_bid': '0.0000347',
  'base_volume': '0',
  'quote_volume': '0',
  'is_frozen': Fal

In [56]:
float([data['last_price'] for coin_pair, data in cob.market.get_stats()['result'].items() if data['id'] == 'BTC-USDT'][0])

2018-08-10 07:41:37 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.market fetch "get_stats"


6366.6

# Limit Order Exploration

In [26]:
cob.market.get_tickers('BTC-USDT')#['result']['ticker']['last_trade_price']

2018-09-08 21:22:27 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_tickers"


{'success': True,
 'result': {'ticker': {'trading_pair_id': 'BTC-USDT',
   'timestamp': 1536456120000,
   '24h_high': '6476.9',
   '24h_low': '6124.8',
   '24h_open': '6422.7',
   '24h_volume': '172.34555137999993',
   'last_trade_price': '6184.3',
   'highest_bid': '6181.3',
   'lowest_ask': '6191.6'}}}

In [82]:
df = pd.DataFrame(cob.market.get_trades('BTC-USDT')['result']['trades'])
df['unix'] = df['timestamp']/1000
df['datetime'] = pd.to_datetime(df['unix'], unit='s')
df

2018-09-10 07:44:58 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_trades"


,id,maker_side,price,size,timestamp,trading_pair_id,unix,datetime
0,665de971-6437-49c5-9c02-040a0eb947ae,ask,6323.2,0.1449737,1536579839563,BTC-USDT,1.536580e+09,2018-09-10 11:43:59.563
1,c89d50fb-35d1-4304-823c-5adc1ed174cf,ask,6302.7,0.00778734,1536579805455,BTC-USDT,1.536580e+09,2018-09-10 11:43:25.455
2,972eda7e-0160-4a1b-8f6d-d7b540d75d91,ask,6298.2,0.005122,1536579781252,BTC-USDT,1.536580e+09,2018-09-10 11:43:01.252
3,5d59a20e-9e8c-4b10-8525-7b99cb199055,bid,6292.4,0.0240639,1536579729118,BTC-USDT,1.536580e+09,2018-09-10 11:42:09.118
4,c285dbde-5bcd-4243-b80c-d811811f3dbd,bid,6293.5,0.03706561,1536579729117,BTC-USDT,1.536580e+09,2018-09-10 11:42:09.117
5,74e1753c-e2d3-47eb-9e81-a3fb95608cd8,bid,6296.5,0.08108315,1536579729117,BTC-USDT,1.536580e+09,2018-09-10 11:42:09.117
6,fb87bf17-d6c1-4def-858c-2dfcd58f9ab7,bid,6296.5,0.00778734,1536579729117,BTC-USDT,1.536580e+09,2018-09-10 11:42:09.117
7,4eb58d2a-6b1c-4c78-bb89-4d40c4c768f0,bid,6296.5,0.03120483,1536579692729,BTC-USDT,1.536580e+09,2018-09-10 11:41:32.729
8,e6e5dc45-e505-414d-b556-20cbd7883967,bid,6298.2,0.005045,1536579692729,BTC-USDT,1.536580e+09,2018-09-10 11:41:32.729
9,00dd6f04-b691-45f1-a9e7-58776399b420,ask,6303.6,0.00503,1536579683456,BTC-USDT,1.536580e+09,2018-09-10 11:41:23.456


In [83]:
cob.market.get_orderbooks('BTC-USDT')

2018-09-10 07:46:41 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_orderbooks"


{'success': True,
 'result': {'orderbook': {'sequence': 0,
   'bids': [['6303.7', '1', '0.05'],
    ['6302.2', '1', '0.03648147'],
    ['6299.4', '1', '0.03'],
    ['6298.8', '1', '0.03'],
    ['6298.7', '1', '0.03'],
    ['6298.1', '1', '0.20688689'],
    ['6296.4', '1', '0.29393582'],
    ['6296.2', '1', '0.03'],
    ['6295.4', '1', '0.03'],
    ['6295', '1', '0.38911626'],
    ['6293.5', '1', '0.24081539'],
    ['6291.9', '2', '0.67485876'],
    ['6290.8', '1', '0.23450187'],
    ['6290.2', '1', '0.33893343'],
    ['6288.5', '1', '0.37413242'],
    ['6286.8', '1', '0.004'],
    ['6286.7', '1', '0.33952503'],
    ['6285.3', '1', '0.03329028'],
    ['6284.9', '2', '0.55051133'],
    ['6284.4', '1', '0.35768664'],
    ['6283.7', '1', '0.34626265'],
    ['6283.4', '1', '0.3276564'],
    ['6281.9', '1', '0.2758563'],
    ['6281.5', '2', '0.60388685'],
    ['6278.2', '1', '0.22134485'],
    ['6270.4', '1', '0.2501678'],
    ['6260.1', '1', '0.03329028'],
    ['6250', '1', '0.02'],
    ['6

In [12]:
ls = cob.market.get_orderbooks('BTC-USDT', limit=100)['result']['orderbook']['bids']
for l in ls:
    l[1] = float(l[1]) * float(l[2])
    l[2] = []
j = {}
j['bids'] = ls
j['unix_timestamp'] = 123432
j

2018-09-29 12:51:15 chriss-mbp.lan cobinhood-api[4256] DEBUG cobinhood_api.http.market fetch "get_orderbooks"


{'bids': [['6554.4', 0.04266919, []],
  ['6552.9', 0.039188, []],
  ['6546.3', 0.03, []],
  ['6545.3', 0.03, []],
  ['6544.4', 0.03, []],
  ['6544', 0.03, []],
  ['6543.1', 0.03, []],
  ['6541.1', 0.24176414, []],
  ['6536.9', 2.88287952, []],
  ['6535.8', 1.1466097, []],
  ['6534.8', 0.04474989, []],
  ['6534.2', 0.26762603, []],
  ['6533', 0.36368642, []],
  ['6528.4', 0.33604076, []],
  ['6523.8', 0.24950979, []],
  ['6518.9', 0.3235175, []],
  ['6508.5', 0.04474989, []],
  ['6505', 0.24, []],
  ['6499.2', 0.2, []],
  ['6486.4', 0.004, []],
  ['6482.3', 0.04474989, []],
  ['6475.3', 0.0055759, []],
  ['6451', 0.03441823, []],
  ['6434.7', 0.0045, []],
  ['6425', 0.0153794, []],
  ['6420', 0.01933105, []],
  ['6400', 1.5246, []],
  ['6390', 0.02, []],
  ['6380', 0.5, []],
  ['6357.2', 0.005, []],
  ['6351', 0.03495015, []],
  ['6350', 0.0219, []],
  ['6330', 0.00599618, []],
  ['6323.5', 0.05, []],
  ['6310', 0.021, []],
  ['6300', 0.72511028, []],
  ['6298.5', 0.00673343, []],
  ['6

### Limit Order Delay
How competitive must our trades be to go through quickly?

In [84]:
def get_top_bids():
    order_book = cob.market.get_orderbooks('BTC-USDT')
    bids = order_book['result']['orderbook']['bids']
    df = pd.DataFrame(bids, columns=['price','qty','volume']).sort_values('price',ascending=False)
    df['price'] = pd.to_numeric(df['price'])
    df['qty'] = pd.to_numeric(df['qty'])
    df['volume'] = pd.to_numeric(df['volume'])
    return df

def get_top_bid():
    top_bids_df = get_top_bids()
    return top_bids_df['price'][0]

def get_top_asks():
    order_book = cob.market.get_orderbooks('BTC-USDT')
    asks = order_book['result']['orderbook']['asks']
    df = pd.DataFrame(asks, columns=['price','qty','volume']).sort_values('price',ascending=True)
    df['price'] = pd.to_numeric(df['price'])
    df['qty'] = pd.to_numeric(df['qty'])
    df['volume'] = pd.to_numeric(df['volume'])
    return df

def get_top_ask():
    top_asks_df = get_top_asks()
    return top_asks_df['price'][0]

In [13]:
def get_orderbook(coin_pair):
    order_book = cob.market.get_orderbooks(coin_pair)
    unix_timestamp = int(time.time())
    # bids df
    orderbook_bids = order_book['result']['orderbook']['bids']
    bids_df = pd.DataFrame(orderbook_bids, columns=['price','qty','volume']).sort_values('price',ascending=False)
    bids_df['price'] = pd.to_numeric(bids_df['price'])
    bids_df['qty'] = pd.to_numeric(bids_df['qty'])
    bids_df['volume'] = pd.to_numeric(bids_df['volume'])
    bids_df['order_type'] = 'bid'
    bids_df.insert(0, 'order_position', range(1, 1 + len(bids_df.index)))
    bids_df['unix_timestamp'] = unix_timestamp
    # asks df
    orderbook_asks = order_book['result']['orderbook']['asks']
    asks_df = pd.DataFrame(orderbook_asks, columns=['price','qty','volume']).sort_values('price',ascending=False)
    asks_df['price'] = pd.to_numeric(asks_df['price'])
    asks_df['qty'] = pd.to_numeric(asks_df['qty'])
    asks_df['volume'] = pd.to_numeric(asks_df['volume'])
    asks_df['order_type'] = 'ask'
    asks_df.insert(0, 'order_position', range(1, 1 + len(asks_df.index)))
    asks_df['unix_timestamp'] = unix_timestamp
    return bids_df, asks_df

In [14]:
bids, asks = get_orderbook('BTC-USDT')
bids

2018-10-01 06:57:57 Chriss-MacBook-Pro.local cobinhood-api[4256] DEBUG cobinhood_api.http.market fetch "get_orderbooks"


,order_position,price,qty,volume,order_type,unix_timestamp
0,1,6600.6,1,0.083400,bid,1538391477
1,2,6600.0,1,0.500000,bid,1538391477
2,3,6576.6,1,0.204108,bid,1538391477
3,4,6576.5,1,0.259034,bid,1538391477
4,5,6575.3,1,0.231243,bid,1538391477
5,6,6574.2,1,0.331559,bid,1538391477
6,7,6572.2,1,0.212554,bid,1538391477
7,8,6571.9,1,0.059168,bid,1538391477
8,9,6570.1,1,0.201436,bid,1538391477
9,10,6569.0,1,0.242566,bid,1538391477


In [16]:
# features
bids['price_var'] = bids['price'].var()
bids

,order_position,price,qty,volume,order_type,unix_timestamp,price_var
0,1,6600.6,1,0.083400,bid,1538391477,17539.694922
1,2,6600.0,1,0.500000,bid,1538391477,17539.694922
2,3,6576.6,1,0.204108,bid,1538391477,17539.694922
3,4,6576.5,1,0.259034,bid,1538391477,17539.694922
4,5,6575.3,1,0.231243,bid,1538391477,17539.694922
5,6,6574.2,1,0.331559,bid,1538391477,17539.694922
6,7,6572.2,1,0.212554,bid,1538391477,17539.694922
7,8,6571.9,1,0.059168,bid,1538391477,17539.694922
8,9,6570.1,1,0.201436,bid,1538391477,17539.694922
9,10,6569.0,1,0.242566,bid,1538391477,17539.694922


In [17]:
asks['price_var'] = asks['price'].var()
asks

,order_position,price,qty,volume,order_type,unix_timestamp,price_var
49,1,7250.0,1,0.010000,ask,1538391477,30711.086322
48,2,7200.0,1,0.038164,ask,1538391477,30711.086322
47,3,7150.0,1,0.500000,ask,1538391477,30711.086322
46,4,7100.0,1,1.000000,ask,1538391477,30711.086322
45,5,7055.0,1,0.004000,ask,1538391477,30711.086322
44,6,7050.0,1,0.200000,ask,1538391477,30711.086322
43,7,7010.0,1,0.004000,ask,1538391477,30711.086322
42,8,7000.0,6,0.227864,ask,1538391477,30711.086322
41,9,6910.0,1,0.004000,ask,1538391477,30711.086322
40,10,6900.0,2,0.304000,ask,1538391477,30711.086322


In [24]:
import numpy
import math

def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = numpy.average(values, weights=weights)
    # Fast and numerically precise:
    variance = numpy.average((values-average)**2, weights=weights)
    return (average, math.sqrt(variance))

print(asks['price'].std())
print(weighted_avg_and_std(asks['price'], asks['volume']))
print(bids['price'].std())
print(weighted_avg_and_std(bids['price'], bids['volume']))

# differences in....
## weighted avg
## weighted std dev
## avg price out of $x volume (at $1000, $5000)
## 

175.24578831586504
(6744.083168749946, 176.51965364047695)
132.4375132749365
(6489.135110991732, 130.99827531608693)


In [29]:
bids['dollar_volume'] = bids['price'] * bids['volume']
bids['cumsum_dollar_dollar_bills'] = bids['dollar_volume'].cumsum()
bids

,order_position,price,qty,volume,order_type,unix_timestamp,price_var,dollar_volume,cumulative_dollar_volume,cumsum_dollar_dollar_bills
0,1,6600.6,1,0.083400,bid,1538391477,17539.694922,550.490040,550.490040,550.490040
1,2,6600.0,1,0.500000,bid,1538391477,17539.694922,3300.000000,3850.490040,3850.490040
2,3,6576.6,1,0.204108,bid,1538391477,17539.694922,1342.336673,5192.826713,5192.826713
3,4,6576.5,1,0.259034,bid,1538391477,17539.694922,1703.537233,6896.363945,6896.363945
4,5,6575.3,1,0.231243,bid,1538391477,17539.694922,1520.494005,8416.857950,8416.857950
5,6,6574.2,1,0.331559,bid,1538391477,17539.694922,2179.732351,10596.590301,10596.590301
6,7,6572.2,1,0.212554,bid,1538391477,17539.694922,1396.947859,11993.538160,11993.538160
7,8,6571.9,1,0.059168,bid,1538391477,17539.694922,388.848348,12382.386508,12382.386508
8,9,6570.1,1,0.201436,bid,1538391477,17539.694922,1323.452627,13705.839135,13705.839135
9,10,6569.0,1,0.242566,bid,1538391477,17539.694922,1593.413229,15299.252364,15299.252364


In [33]:
def weighted_avg(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    return numpy.average(values, weights=weights)
weighted_avg(bids[bids['cumsum_dollar_dollar_bills'] < 5000]['price'], bids[bids['cumsum_dollar_dollar_bills'] < 5000]['volume'])
weighted_avg(bids[bids['cumsum_dollar_dollar_bills'] < 10000]['price'], bids[bids['cumsum_dollar_dollar_bills'] < 10000]['volume'])

6600.085773054508

In [38]:
(weighted_avg(bids[bids['cumsum_dollar_dollar_bills'] < 10000]['price'], bids[bids['cumsum_dollar_dollar_bills'] < 10000]['volume']) - bids['price'][0]) / bids['price'][0]

-0.0020502086990188515

In [35]:
)

SyntaxError: invalid syntax (<ipython-input-35-a35077a93b18>, line 1)

In [ ]:
get_top_bids()

In [54]:
get_top_asks()

2018-09-10 07:07:03 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_orderbooks"


,price,qty,volume
0,6290.5,1,0.578685
1,6291.5,1,0.270498
2,6292.7,1,0.041096
3,6300.3,1,0.030000
4,6300.4,1,0.030000
5,6300.6,1,0.030000
6,6303.5,1,0.039000
7,6303.7,1,0.039000
8,6315.5,1,0.245818
9,6317.4,2,0.655808


## BID ASK TRADER

In [88]:
# Weighted average spread of top n that are 
top_bids_df = get_top_bids()
top_5_bids_df = top_bids_df.head(5)
weighted_average_bid_price = sum(top_5_bids_df['price']*top_5_bids_df['volume']*top_5_bids_df['qty']
                            ) / sum(top_5_bids_df['volume']*top_5_bids_df['qty'])
print(f"weighted average bid: {weighted_average_bid_price}")
top_bid_price = top_bids_df['price'][0]
print(f"Top bid price: {top_bid_price}")

# Trade
bid_trade_data = {
            "trading_pair_id": "BTC-USDT",
            "side": "bid",
            "type": "limit",
            "price": f"{top_bid_price}",
            "size": ".05"
        }
buy_order_start_time = time.time()
buy_order = cob.trading.post_orders(bid_trade_data)
# Check for failure 
print(buy_order)
if not buy_order['success']:
    raise ValueError(f"The buy order returned unsuccessfully! Payload: {buy_order}")
buy_order_id = buy_order['result']['order']['id']
buy_order_state = 'sent'
# Wait for completion
while buy_order_state != 'filled':
    time.sleep(1)
    buy_order = cob.trading.get_orders(buy_order_id)
    if not buy_order['success']:
        raise ValueError(f"The buy order returned unsuccessfully! Payload: {buy_order}")
    buy_order_state = buy_order['result']['order']['state']
    if buy_order_state == 'cancelled':
        raise ValueError("The buy order was cancelled. Investigate if this behavior should be handled going forward.")
    current_bid_price = get_top_bid()
    if current_bid_price > top_bid_price:
        top_bid_price = current_bid_price
        print(f"Current top bid: {current_bid_price} at {time.time() - buy_order_start_time} sec since bid")
    #print(buy_order)
buy_order_fill_time = time.time()
buy_fill_latency_seconds = buy_order_fill_time - buy_order_start_time
print(f"Buying took {buy_fill_latency_seconds}. Info: {buy_order}")
final_buy_price = buy_order['result']['order']['eq_price']


top_asks_df = get_top_asks()
top_5_asks_df = top_asks_df.head(5)
weighted_average_ask_price = sum(top_5_asks_df['price']*top_5_asks_df['volume']*top_5_asks_df['qty']
                            ) / sum(top_5_asks_df['volume']*top_5_asks_df['qty'])
print(f"weighted average ask: {weighted_average_ask_price}")
top_ask_price = top_asks_df['price'][0]
print(f"Top ask price: {top_ask_price}")

# Trade asks
ask_trade_data = {
            "trading_pair_id": "BTC-USDT",
            "side": "ask",
            "type": "limit",
            "price": f"{top_ask_price}",
            "size": ".05"
        }
sell_order_start_time = time.time()
sell_order = cob.trading.post_orders(ask_trade_data)
# Check for failure 
print(sell_order)
if not sell_order['success']:
    raise ValueError(f"The sell order returned unsuccessfully! Payload: {sell_order}")
sell_order_id = sell_order['result']['order']['id']
sell_order_state = 'sent'
# Wait for completion
while sell_order_state != 'filled':
    time.sleep(1)
    sell_order = cob.trading.get_orders(sell_order_id)
    if not sell_order['success']:
        raise ValueError(f"The sell order returned unsuccessfully! Payload: {sell_order}")
    sell_order_state = sell_order['result']['order']['state']
    if sell_order_state == 'cancelled':
        raise ValueError("The sell order was cancelled. Investigate if this behavior should be handled going forward.")
    current_ask_price = get_top_ask()
    if current_ask_price > top_ask_price:
        top_ask_price = current_ask_price
        print(f"Current top ask: {current_ask_price} at {time.time() - ask_order_start_time} sec since ask")
    #print(sell_order)
sell_order_fill_time = time.time()
sell_fill_latency_seconds = sell_order_fill_time - sell_order_start_time
print(f"Selling took {sell_order_fill_time}. Info: {sell_order}")

final_sell_price = sell_order['result']['order']['eq_price']

print(f"We made: ${final_sell_price - final_buy_price}!!! Fuck yea.")

2018-09-12 06:14:26 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_orderbooks"
2018-09-12 06:14:27 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.trading fetch "post_orders"


weighted average bid: 6239.242428797529
Top bid price: 6239.3
{'success': True, 'result': {'order': {'id': '71c91b78-ba77-47e6-844e-9dc6ad66e07d', 'trading_pair_id': 'BTC-USDT', 'side': 'bid', 'type': 'limit', 'price': '6239.3', 'size': '0.05', 'filled': '0', 'state': 'queued', 'timestamp': 1536747268608, 'eq_price': '0', 'completed_at': None, 'source': 'exchange'}}}


2018-09-12 06:14:29 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.trading fetch "get_orders"
2018-09-12 06:14:30 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_orderbooks"


Current top bid: 6239.3


2018-09-12 06:14:31 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.trading fetch "get_orders"
2018-09-12 06:14:31 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_orderbooks"


...

Current top ask 6238.5


2018-09-12 07:30:47 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.trading fetch "get_orders"
2018-09-12 07:30:47 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.market fetch "get_orderbooks"


Current top ask 6238.7


2018-09-12 07:30:48 Chriss-MacBook-Pro.local cobinhood-api[1699] DEBUG cobinhood_api.http.trading fetch "get_orders"


ValueError: The sell order was cancelled. Investigate if this behavior should be handled going forward.